## Analyse grammaticale avec NLTK et Spacy
On présente dans ce notebook une comparaison des fonctions de base de NLTK et Spacy pour effectuer une analyse grammaticale (part-of-speech tagging).

## NLTK
Tout d'abord, NLTK offrent plusieurs implémentations de POS taggers (Stanford, transformation-based, n-grams, etc.). Toutefois, on se limite ici à celui offert par la fonction pos-tag. L'analyse est faite au niveau de la phrase et on doit faire une tokeniser la phrase au préalable. Les étiquettes correspondent à celles du Penn Treebank.

In [10]:
import nltk

sent = "This is a test."
tokens = nltk.word_tokenize(sent)
tagged_tokens = nltk.pos_tag(tokens)
for token, tag in tagged_tokens:
    print(token, tag)

AttributeError: partially initialized module 'nltk' has no attribute 'download' (most likely due to a circular import)

In [ ]:
!pip uninstall nltk
!pip install -U nltk

In [6]:
nltk.download()

NameError: name 'nltk' is not defined